# Standard Bank Hackathon

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
from mlxtend.preprocessing import TransactionEncoder
from mpl_toolkits.mplot3d import Axes3D
import networkx as nx
import seaborn as sns
import re

/Users/katerine/opt/anaconda3/envs/NLP_Assignments/lib/python3.10/site-packages/seaborn/rcmod.py:82: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if LooseVersion(mpl.__version__) >= "3.0":
/Users/katerine/opt/anaconda3/envs/NLP_Assignments/lib/python3.10/site-packages/setuptools/_distutils/version.py:346: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  other = LooseVersion(other)


## Basket analysis

In [2]:
def item_rules(rules, item_name):
    rules_for_item = rules[rules['consequents'].astype(str).str.contains(item_name)]
    rules_for_item = rules_for_item.sort_values(by=['lift'],ascending = [False]).reset_index(drop=True)
    display(rules_for_item.head())

### Restaurant 1

In [3]:
orders_1 = pd.read_csv('data/restaurant-1-orders.csv')
products_1 = pd.read_csv('data/restaurant-1-products-price.csv')

focus_products_1 = pd.read_csv('data/focus_products.csv', header=None)
focus_products_1 = list(focus_products_1[0])

In [4]:
display(orders_1.head())

,Order Number,Order Date,Item Name,Quantity,Product Price,Total products
0,16118,03/08/2019 20:25,Plain Papadum,2,0.80,6
1,16118,03/08/2019 20:25,King Prawn Balti,1,12.95,6
2,16118,03/08/2019 20:25,Garlic Naan,1,2.95,6
3,16118,03/08/2019 20:25,Mushroom Rice,1,3.95,6
4,16118,03/08/2019 20:25,Paneer Tikka Masala,1,8.95,6


In [5]:
display(products_1.head())

,Item Name,Product Price
0,Mint Sauce,0.5
1,Lime Pickle,0.5
2,Mango Chutney,0.5
3,Red Sauce,0.5
4,Onion Chutney,0.5


In [6]:
orders = orders_1[['Order Number', 'Item Name']].rename(columns={'Order Number':'order_id', 'Item Name':'item_name'})

In [7]:
orders = orders[orders['item_name'].isin(focus_products_1)]

In [8]:
orders.item_name = orders.item_name.transform(lambda x: [x])
orders = orders.groupby(['order_id']).sum()['item_name'].reset_index(drop=True)

encoder = TransactionEncoder()
transactions = pd.DataFrame(encoder.fit(orders).transform(orders), columns=encoder.columns_)
display(transactions.head())

,Aloo Chaat,Aloo Gobi,Aloo Methi,Baingan Hari Mirch,Bengal Fish Biryani,Bengal Fish Karahi,Bengal Fry Fish,Bengal King Prawn,Bengal Salad,Bhindi Bhajee,...,Tandoori Mixed Grill,Tandoori Roti,Tarka Dall,Vegetable Balti,Vegetable Biryani,Vegetable Karahi,Vegetable Mysore,Vegetable Rice,Vegetable Roll,Vindaloo Sauce
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [9]:
frequent_itemsets = apriori(transactions, min_support=0.2, use_colnames=True)
frequent_itemsets['length'] = frequent_itemsets['itemsets'].apply(lambda x: len(x))
frequent_itemsets

,support,itemsets,length
0,0.353474,(Pilau Rice),1
1,0.280922,(Plain Naan),1
2,0.269242,(Plain Papadum),1


In [10]:
frequent_itemsets = apriori(transactions, min_support=0.05, use_colnames=True)
frequent_itemsets['length'] = frequent_itemsets['itemsets'].apply(lambda x: len(x))
print(frequent_itemsets[frequent_itemsets['length'] >= 2])

     support                                    itemsets  length
20  0.065514                   (Pilau Rice, Bombay Aloo)       2
21  0.085355          (Chicken Tikka Masala, Pilau Rice)       2
22  0.064615          (Chicken Tikka Masala, Plain Naan)       2
23  0.060048       (Plain Papadum, Chicken Tikka Masala)       2
24  0.081686                   (Pilau Rice, Garlic Naan)       2
25  0.053309                (Plain Papadum, Garlic Naan)       2
26  0.053908                 (Mango Chutney, Mint Sauce)       2
27  0.073750                 (Pilau Rice, Mango Chutney)       2
28  0.052411                 (Mango Chutney, Plain Naan)       2
29  0.113357              (Plain Papadum, Mango Chutney)       2
30  0.081087                 (Plain Papadum, Mint Sauce)       2
31  0.079665                  (Pilau Rice, Onion Bhajee)       2
32  0.053459                  (Plain Naan, Onion Bhajee)       2
33  0.066637               (Plain Papadum, Onion Bhajee)       2
34  0.058476             

In [11]:
frequent_itemsets = apriori(transactions, min_support=6/len(orders), use_colnames=True, max_len=2)
rules = association_rules(frequent_itemsets, metric="lift",  min_threshold=1.5)
display(rules.head())
print("Rules identified: ", len(rules))

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(Baingan Hari Mirch),(Aloo Chaat),0.014675,0.021264,0.000524,0.035714,1.679577,0.000212,1.014986
1,(Aloo Chaat),(Baingan Hari Mirch),0.021264,0.014675,0.000524,0.024648,1.679577,0.000212,1.010225
2,(Aloo Chaat),(Chana Masala),0.021264,0.028377,0.001123,0.052817,1.861273,0.000520,1.025803
3,(Chana Masala),(Aloo Chaat),0.028377,0.021264,0.001123,0.039578,1.861273,0.000520,1.019069
4,(Aloo Chaat),(Chicken Chaat),0.021264,0.025232,0.001273,0.059859,2.372341,0.000736,1.036832


Rules identified:  2752


In [12]:
item_rules(rules, 'Plain Papadum')

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(Red Sauce),(Plain Papadum),0.049790,0.269242,0.037960,0.762406,2.831673,0.024555,3.075657
1,(Onion Chutney),(Plain Papadum),0.077269,0.269242,0.058476,0.756783,2.810788,0.037672,3.004550
2,(Mint Sauce),(Plain Papadum),0.109539,0.269242,0.081087,0.740260,2.749419,0.051595,2.813417
3,(Mango Chutney),(Plain Papadum),0.154912,0.269242,0.113357,0.731754,2.717829,0.071649,2.724212
4,(Dhansak Sauce),(Plain Papadum),0.006289,0.269242,0.004567,0.726190,2.697164,0.002874,2.668854


In [13]:
item_rules(rules, 'Butter Chicken')

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(Chicken Achar),(Butter Chicken),0.002770,0.073375,0.000599,0.216216,2.946718,0.000396,1.182245
1,(Korma - Lamb),(Butter Chicken),0.004268,0.073375,0.000749,0.175439,2.390977,0.000436,1.123779
2,(Prawn Balti),(Butter Chicken),0.002695,0.073375,0.000449,0.166667,2.271429,0.000251,1.111950
3,(Lamb Chilli Garlic),(Butter Chicken),0.005016,0.073375,0.000824,0.164179,2.237527,0.000456,1.108640
4,(Paneer Tikka Sizzler),(Butter Chicken),0.005016,0.073375,0.000824,0.164179,2.237527,0.000456,1.108640


### Restaurant 2

In [14]:
orders_2 = pd.read_csv('data/restaurant-2-orders.csv')
products_2 = pd.read_csv('data/restaurant-2-products-price.csv')

focus_products_2 = pd.read_csv('data/restaurant_2_focus_products.csv', header=None)
focus_products_2 = list(focus_products_2[0])

In [15]:
display(orders_2.head())

,Order ID,Order Date,Item Name,Quantity,Product Price,Total products
0,25583,03/08/2019 21:58,Tandoori Mixed Grill,1,11.95,12
1,25583,03/08/2019 21:58,Madras Sauce,1,3.95,12
2,25583,03/08/2019 21:58,Mushroom Rice,2,3.95,12
3,25583,03/08/2019 21:58,Garlic Naan,1,2.95,12
4,25583,03/08/2019 21:58,Paratha,1,2.95,12


In [16]:
display(products_2.head())

,Item Name,Product Price
0,Onion Chutney,0.5
1,Mint Sauce,0.5
2,Mango Chutney,0.5
3,Red Sauce,0.5
4,Lime Pickle,0.5


In [17]:
orders = orders_2[['Order ID', 'Item Name']].rename(columns={'Order ID':'order_id', 'Item Name':'item_name'})

In [18]:
orders = orders[orders['item_name'].isin(focus_products_2)]

In [19]:
orders.item_name = orders.item_name.transform(lambda x: [x])
orders = orders.groupby(['order_id']).sum()['item_name'].reset_index(drop=True)

encoder = TransactionEncoder()
transactions = pd.DataFrame(encoder.fit(orders).transform(orders), columns=encoder.columns_)
display(transactions.head())

,Achar Chicken,Achar Lamb,Aloo Brinjal,Aloo Chaat,Aloo Dupiaza,Aloo Gobi,Aloo Mithy,Aloo Peas,Bangon Hari Mirch,Bhindi Bhajee,...,Vegetable Dansak,Vegetable Jalfrezi,Vegetable Karahi,Vegetable Korma,Vegetable Masala,Vegetable Rice,Vegetable Samba,Vegetable Samosa,Vindaloo,Vindaloo Sauce
0,False,False,False,False,False,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [20]:
frequent_itemsets = apriori(transactions, min_support=0.2, use_colnames=True)
frequent_itemsets['length'] = frequent_itemsets['itemsets'].apply(lambda x: len(x))
frequent_itemsets

,support,itemsets,length
0,0.210440,(Bombay Aloo),1
1,0.340487,(Naan),1
2,0.426692,(Pilau Rice),1
3,0.327845,(Plain Papadum),1


In [21]:
frequent_itemsets = apriori(transactions, min_support=0.075, use_colnames=True)
frequent_itemsets['length'] = frequent_itemsets['itemsets'].apply(lambda x: len(x))
frequent_itemsets[frequent_itemsets['length'] >= 2]

,support,itemsets,length
19,0.095177,"(Bombay Aloo, Naan)",2
20,0.112459,"(Pilau Rice, Bombay Aloo)",2
21,0.080750,"(Plain Papadum, Bombay Aloo)",2
22,0.077539,"(Chicken Tikka Masala, Naan)",2
23,0.113836,"(Chicken Tikka Masala, Pilau Rice)",2
24,0.092628,"(Pilau Rice, Garlic Naan)",2
25,0.087429,"(Pilau Rice, Mango Chutney)",2
26,0.149980,"(Plain Papadum, Mango Chutney)",2
27,0.099765,"(Plain Papadum, Mint Sauce)",2
28,0.178426,"(Pilau Rice, Naan)",2


In [22]:
frequent_itemsets = apriori(transactions, min_support= 6/len(orders), use_colnames=True, max_len=2)
rules = association_rules(frequent_itemsets, metric="lift",  min_threshold = 1.5)
display(rules.head())
print('Rules identified: ', len(rules))

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(Achar Chicken),(Achar Lamb),0.006474,0.003925,0.001784,0.275591,70.207588,0.001759,1.375016
1,(Achar Lamb),(Achar Chicken),0.003925,0.006474,0.001784,0.454545,70.207588,0.001759,1.821464
2,(Achar Chicken),(Aloo Chaat),0.006474,0.011521,0.000306,0.047244,4.100620,0.000231,1.037494
3,(Aloo Chaat),(Achar Chicken),0.011521,0.006474,0.000306,0.026549,4.100620,0.000231,1.020622
4,(Achar Chicken),(Aloo Gobi),0.006474,0.040987,0.000408,0.062992,1.536883,0.000142,1.023485


Rules identified:  3688


In [23]:
item_rules(rules, 'Pilau Rice')

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(Lal Mirch Chicken),(Pilau Rice),0.002906,0.426692,0.001988,0.684211,1.603521,0.000748,1.815474
1,(Paneer Dansak),(Pilau Rice),0.001937,0.426692,0.001325,0.684211,1.603521,0.000499,1.815474
2,(Lamb Tikka Pasanda),(Pilau Rice),0.003263,0.426692,0.002192,0.671875,1.574612,0.000800,1.747223
3,(Chicken Tikka Masala),(Pilau Rice),0.177763,0.426692,0.113836,0.640379,1.500796,0.037985,1.594197
